**This program will parse your data of midi files into data readable by the ai**

Note that before running this, make sure you have a dataset of midi files, I would recomend atleast ~10,000 for decent results, but that is assumming they are all imported correctly, so probably more like 80,000.

For finetuning, you need signifigantly less. Probably 100-1000?

Import

In [ ]:
!pip install py_midicsv
!pip install mido

In [ ]:
!rm -rf /content/music/

**Upload your zip file of midi files (all in root of zip) and rename to music.zip also create a folder under the same name in the content folder**
Make sure that any midi files arent in a folder of a folder, they wont be found

In [ ]:
!unzip /content/music.zip -d /content/music/

In [ ]:
!ls -lh

total 8.0K
drwx------ 5 root root 4.0K Sep  3 12:46 drive
drwxr-xr-x 1 root root 4.0K Aug 31 13:25 sample_data


In [ ]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
# Fogor to scan MID
import py_midicsv as pm
import os
import re


midi_dir = "/content/MIDI/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith(('.mid', '.MID'))]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  out_instrument = "%"
  match instrument:
    case 2:
      out_instrument = "%"
    case 3:
      out_instrument = "^"
    case 4:
      out_instrument = "&"
    case 5:
      out_instrument = "*"
    case 6:
      out_instrument = "("
    case _:
      out_instrument = "("

  if(int(instrument) in tracksThatAreDrums):
    out_instrument = ")"

  dynamic = "$"
  if(velocity > 108):
    dynamic = "$"
  elif velocity > 74:
    dynamic = "#"
  elif velocity > 40:
    dynamic = "@"
  else:
    dynamic = "!"

  return [start, dynamic, length, out_instrument, note]

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
numNotesInCsv = 0
num_filesSoFar = 0
current_is_mid_1 = True
for midi_file in midi_files:
  csv_parsed = []
  midiSubdivision = 1
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    notesPlaying = []
    tracksThatAreDrums = []

    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Control_c":
          if(current_is_mid_1):
            # if 9th channel, then it is drums according to midi
            if(int(split_csv_item[3] )== 9):
                if(not (int(split_csv_item[0]) in tracksThatAreDrums)):
                    tracksThatAreDrums.append(int(split_csv_item[0]))
        case "Program_c":
          if(not current_is_mid_1):
            if(int(split_csv_item[4] )== 127 or int(split_csv_item[4] )== 0):
                if(not (int(split_csv_item[3]) in tracksThatAreDrums)):
                    tracksThatAreDrums.append(int(split_csv_item[3]))

        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(current_is_mid_1):
              if(notesPlaying[i][4] == split_csv_item[4]):
                temp = get_note_event(int(notesPlaying[i][1] * midiSubdivision), split_csv_item[4], split_csv_item[0], int((split_csv_item[1] * midiSubdivision) - (notesPlaying[i][1] * midiSubdivision)), notesPlaying[i][5])
                csv_parsed.append(temp)
                lastNoteStart = notesPlaying[i][1] * midiSubdivision
                notesPlaying.pop(i)
                i-=1
              i+=1
            else:
              if(notesPlaying[i][4] == split_csv_item[4]):
                temp = get_note_event(int(notesPlaying[i][1] * midiSubdivision), split_csv_item[4], split_csv_item[3], int((split_csv_item[1] * midiSubdivision) - (notesPlaying[i][1] * midiSubdivision)), notesPlaying[i][5])
                csv_parsed.append(temp)
                lastNoteStart = notesPlaying[i][1] * midiSubdivision
                notesPlaying.pop(i)
                i-=1
              i+=1
        case "Header":
          if(int(split_csv_item[3]) == 0):
            current_is_mid_1 = False
          else:
            current_is_mid_1 = True
          midiSubdivision = 480 / float(split_csv_item[5])

  except Exception as e:
    print(e)

  # Change to be delta time instead of abs and sort time
  csv_parsed = sorted(csv_parsed, key=lambda x: x[0])
  for i in range(len(csv_parsed) - 1):
    csv_item_copy = csv_parsed[i].copy()
    if(i != 0):
      csv_item_copy[0] = csv_parsed[i][0] - csv_parsed[i - 1][0]

    # Also concat to csv_parsed
    for j in csv_item_copy:
      csv_string += str(j)
    csv_string += "|"
    numNotesInCsv+=1
    if(numNotesInCsv % 4000 == 0):
      csv_string += "\n"
  csv_string += "\n"
  num_filesSoFar+=1
  if(num_filesSoFar % 20000 == 0):
    with open("convertedData"+str(num_filesSoFar)+".txt", "w") as f:
      f.writelines(csv_string)
  if(num_filesSoFar % 50 == 0):
    print(num_filesSoFar)

#for i in range(0, len(csv_string), 1800):
#  csv_string[i] = "\n"

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)

with open("/content/drive/MyDrive/bignicedata.txt", "w") as f:
    f.writelines(csv_string)

# Copy the generated data to your google drive

In [ ]:
!cp /content/convertedData.txt /content/drive/MyDrive/